In [1]:
#James Chartouni

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor

from IPython.display import display # Allows the use of display() for DataFrames

import warnings
warnings.filterwarnings('ignore')
from scipy.stats import boxcox
from scipy.special import inv_boxcox

from helpers import *

In [2]:
train = pd.read_csv("data/train_cleaned.csv")
train_orig = pd.read_csv("data/train.csv")
#y_train = train_orig['target'] #TRY A LOG TRANSFORM
y_train = np.log1p(train_orig["target"].values)
#y_train = boxcox(train_orig["target"].values)[0]
#X_train = train.drop(["target", "ID"], axis=1)
X_train = train

print ("Rows: " + str(train.shape[0]) + ", Columns: " + str(train.shape[1]))
print(train.head())

test = pd.read_csv('data/test_cleaned.csv')
#test_X = test.drop(["ID"], axis=1)
print ("Rows: " + str(test.shape[0]) + ", Columns: " + str(test.shape[1]))
test_X = test
test.head()

Rows: 4459, Columns: 1420
   SumValues  SumZeros  20aa07010  dc5a8f1d8  4681de4fd  adf119b9a  0d866c3d7  \
0        103      4628   0.000000        0.0        0.0        0.0        0.0   
1         68      4663  14.603968        0.0        0.0        0.0        0.0   
2         19      4712   0.000000        0.0        0.0        0.0        0.0   
3         23      4708   0.000000        0.0        0.0        0.0        0.0   
4         27      4704  14.508658        0.0        0.0        0.0        0.0   

   bd8f989f1  22ed6dba3  92b13ebba       ...             Std  Q4.95  \
0        0.0        0.0        0.0       ...        2.169096    0.0   
1        0.0        0.0        0.0       ...        1.809385    0.0   
2        0.0        0.0        0.0       ...        0.909701    0.0   
3        0.0        0.0        0.0       ...        0.887781    0.0   
4        0.0        0.0        0.0       ...        1.098374    0.0   

          Q5       Skew       mad        kurt       sem  num

,SumValues,SumZeros,20aa07010,dc5a8f1d8,4681de4fd,adf119b9a,0d866c3d7,bd8f989f1,22ed6dba3,92b13ebba,...,Std,Q4.95,Q5,Skew,mad,kurt,sem,number_of_different,non_zero_count,geometric_mean
0,73,4658,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,1.829730,0.0,20.682969,8.348649,0.438792,69.445898,0.026605,83,85,14.457006
1,11,4720,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.645842,0.0,17.273869,22.865799,0.058110,534.362437,0.009391,21,23,9.579583
2,100,4631,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,2.037257,0.0,18.197537,6.921548,0.576216,46.797912,0.029622,84,112,14.151046
3,121,4610,0.0,0.0,0.0,0.0,0.0,16.854494,0.0,0.0,...,2.510632,0.0,18.420681,6.121881,0.785246,35.733973,0.036505,111,133,16.031065
4,9,4722,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.617740,0.0,18.494941,24.977718,0.050213,631.998291,0.008982,19,21,9.390606


In [3]:
X_train.info()
test_X.info()
y_train

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4459 entries, 0 to 4458
Columns: 1420 entries, SumValues to geometric_mean
dtypes: float64(1416), int64(4)
memory usage: 48.3 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49342 entries, 0 to 49341
Columns: 1420 entries, SumValues to geometric_mean
dtypes: float64(1416), int64(4)
memory usage: 534.6 MB


array([ 17.45309674,  13.3046866 ,  16.11809575, ...,  14.84513033,
        16.11809575,  16.81124288])

In [4]:
#### Check if there are any NULL values in Train Data
print("Total Train Features with NaN Values = " + str(X_train.columns[X_train.isnull().sum() != 0].size))
if (X_train.columns[X_train.isnull().sum() != 0].size):
    print("Features with NaN => {}".format(list(X_train.columns[X_train.isnull().sum() != 0])))
    X_train[X_train.columns[X_train.isnull().sum() != 0]].isnull().sum().sort_values(ascending = False)

Total Train Features with NaN Values = 0


In [5]:
#normalize and scale the data
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler

#minMax_train = MinMaxScaler().fit_transform(X_train.values)
#standardScaler_train = StandardScaler().fit_transform(X_train.values)
#robustScaler_train = RobustScaler().fit_transform(X_train.values)
#maxAbsScaler_train = MaxAbsScaler().fit_transform(X_train.values)


#minMax_test = MinMaxScaler().fit_transform(test_X.values)
#standardScaler_test = StandardScaler().fit_transform(test_X.values)
#robustScaler_test = RobustScaler().fit_transform(test_X.values)
#maxAbsScaler_test = MaxAbsScaler().fit_transform(test_X.values)


In [6]:
def lgbm_auto(data, y_train, test_data):
      
    train_X, val_X, train_y, val_y = train_test_split(data, y_train, test_size = 0.2, random_state = 42)
    
    params = {
        "objective" : "regression",
        "metric" : 'rmse',
        "num_leaves" : 40,
        "learning_rate" : 0.001,
        "bagging_fraction" : 0.7,
        "feature_fraction" : 0.6,
        "bagging_frequency" : 6,
        "bagging_seed" : 42,
        "verbosity" : -1,
        "seed": 42,
        "n_threads": -1
    }
    
    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    evals_result = {}
    model = lgb.train(params, lgtrain, 5000, 
                      valid_sets=[lgtrain, lgval],
                      early_stopping_rounds=100, 
                      verbose_eval=50, 
                      evals_result=evals_result)
    
    pred_test_y = np.expm1(model.predict(test_data, num_iteration=model.best_iteration))
    #pred_test_y = inv_boxcox(model.predict(test_data, num_iteration=model.best_iteration))
    return pred_test_y, model, evals_result

In [7]:
'''print("baseline")
pred_test_lgbm, model_lgbm, evals_result_lgbm = lgbm_auto(X_train, y_train, test_X)
print("----------------------------------------------")
print("minMAx")
pred_test_minMax_lgbm, model_minMax_lgbm, evals_result_minMax_lgbm = lgbm_auto(minMax_train, y_train, minMax_test)
print("----------------------------------------------")
print("standardScaler")
pred_test_standardScaler_lgbm, model_standardScaler_lgbm, evals_result_standardScaler_lgbm = lgbm_auto(standardScaler_train, y_train, standardScaler_test)
print("----------------------------------------------")
print("robustScaler")
pred_test_robustScaler_lgbm, model_robustScaler_lgbm, evals_result_robustScaler_lgbm = lgbm_auto(robustScaler_train, y_train, robustScaler_test)
print("----------------------------------------------")'''
#print("maxAbsScaler")
#pred_test_maxAbsScaler_lgbm, model_maxAbsScaler_lgbm, evals_result_maxAbsScaler_lgbm = lgbm_auto(maxAbsScaler_train, y_train, maxAbsScaler_test)
print("vanilla")
pred_test_lgbm, model_lgbm, evals_result_lgbm = lgbm_auto(X_train, y_train, test_X)

vanilla


TypeError: Unknown type of parameter:feval, got:function

In [ ]:
'''# feature importance
print("Features Importance model_minMax...")
gain = model_minMax_lgbm.feature_importance('gain')
featureimp = pd.DataFrame({'feature':model_minMax_lgbm.feature_name(), 
                   'split':model_minMax_lgbm.feature_importance('split'), 
                   'gain':100 * gain / gain.sum()}).sort_values('gain', ascending=False)
print(featureimp[:15])

# feature importance
print("Features Importance model_standardScaler...")
gain = model_standardScaler_lgbm.feature_importance('gain')
featureimp = pd.DataFrame({'feature':model_standardScaler_lgbm.feature_name(), 
                   'split':model_standardScaler_lgbm.feature_importance('split'), 
                   'gain':100 * gain / gain.sum()}).sort_values('gain', ascending=False)
print(featureimp[:15])

# feature importance
print("Features Importance model_robustScaler...")
gain = model_robustScaler_lgbm.feature_importance('gain')
featureimp = pd.DataFrame({'feature':model_robustScaler_lgbm.feature_name(), 
                   'split':model_robustScaler_lgbm.feature_importance('split'), 
                   'gain':100 * gain / gain.sum()}).sort_values('gain', ascending=False)
print(featureimp[:15])'''

# feature importance
print("Features Importance model_maxAbsScaler...")
gain = model_lgbm.feature_importance('gain')
featureimp = pd.DataFrame({'feature':model_lgbm.feature_name(), 
                   'split':model_lgbm.feature_importance('split'), 
                   'gain':100 * gain / gain.sum()}).sort_values('gain', ascending=False)
print(featureimp[:50])

In [ ]:
def xgb_auto(data, y_train, test_data):
      
    train_X, val_X, train_y, val_y = train_test_split(data, y_train, test_size = 0.2, random_state = 42)
    
    params = {'objective': 'reg:linear', 
          'eval_metric': 'rmse',
          'eta': 0.001,
          'max_depth': 10, 
          'subsample': 0.6, 
          'colsample_bytree': 0.6,
          'alpha':0.001,
          'random_state': 42, 
          'silent': True}
    
    tr_data = xgb.DMatrix(train_X, train_y)
    va_data = xgb.DMatrix(val_X, val_y)
    
    watchlist = [(tr_data, 'train'), (va_data, 'valid')]
    
    model_xgb = xgb.train(params, tr_data, 3500, watchlist, maximize=False, early_stopping_rounds = 100, verbose_eval=50)
    
    dtest = xgb.DMatrix(test_data)
    xgb_pred_y = np.expm1(model_xgb.predict(dtest, ntree_limit=model_xgb.best_ntree_limit))
    
    return xgb_pred_y, model_xgb

In [ ]:
#print("minMAx")
#pred_test_minMax_xgb, model_minMax_xgb = xgb_auto(minMax_train, y_train, minMax_test))
#print("----------------------------------------------")
#print("standardScaler")
#pred_test_standardScaler_xgb, model_standardScaler_xgb = xgb_auto(standardScaler_train, y_train, standardScaler_test)
#print("----------------------------------------------")
#print("robustScaler")
#pred_test_robustScaler_xgb, model_robustScaler_xgb = xgb_auto(robustScaler_train, y_train, robustScaler_test)
print("----------------------------------------------")
print("maxAbsScaler")
pred_test_maxAbsScaler_xgb, model_maxAbsScaler_xgb = xgb_auto(maxAbsScaler_train, y_train, maxAbsScaler_test)

In [ ]:
def cat_auto(data, y_train, test_data):
      
    train_X, val_X, train_y, val_y = train_test_split(data, y_train, test_size = 0.2, random_state = 42)
    
    cb_model = CatBoostRegressor(iterations=500,
                             learning_rate=0.05,
                             depth=10,
                             eval_metric='RMSE',
                             random_seed = 42,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 20,
                             od_wait=20)
    
    cb_model.fit(train_X, train_y,
             eval_set=(val_X, val_y),
             use_best_model=True,
             verbose=True)
    
    pred_test_cat = np.expm1(cb_model.predict(test_data))
    
    return pred_test_cat, cb_model

In [ ]:
'''print("minMax")
pred_test_minMax_cat, model_minMax_cat = cat_auto(minMax_train, y_train, minMax_test)
print("----------------------------------------------")
print("standardScaler")
pred_test_standardScaler_cat, model_standardScaler_cat = cat_auto(standardScaler_train, y_train, standardScaler_Test)
print("----------------------------------------------")
print("robustScaler")
pred_test_robustScaler_cat, model_robustScaler_cat = cat_auto(robustScaler_train, y_train, robustScaler_test)
print("----------------------------------------------")'''
print("maxAbsScaler")
pred_test_maxAbsScaler_cat, model_maxAbsScaler_cat = cat_auto(maxAbsScaler_train, y_train, maxAbsScaler_test)

In [ ]:
#LGBM predictions
sub = pd.read_csv('data/sample_submission.csv')

sub_lgb = pd.DataFrame()
sub_lgb["target"] = pred_test_lgbm

sub["target"] = sub_lgb["target"] 

print(sub.head())
sub.to_csv('submit/sub_lgb.csv', index=False)

In [ ]:
#combine predictions
sub = pd.read_csv('data/sample_submission.csv')

sub_lgb = pd.DataFrame()
sub_lgb["target"] = pred_test_maxAbsScaler_lgbm

sub_xgb = pd.DataFrame()
sub_xgb["target"] = pred_test_maxAbsScaler_xgb

sub_cat = pd.DataFrame()
sub_cat["target"] = pred_test_maxAbsScaler_cat

sub["target"] = (sub_lgb["target"] + sub_xgb["target"] + sub_cat["target"])/3


In [ ]:
print(sub.head())
sub.to_csv('submit/sub_lgb_xgb_cat.csv', index=False)